In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 200)

---

**BEGIN CONTENT HERE**

---

Our dataset features two distinct components that we merged together: game-specific features and team statistics. Each game features two opponents, both of which can be joined to the team statistics described below, and one of which is designated the “primary” team in each game (i.e., we predict a win or loss *for that team*). Naturally, we note game outcomes, as well as other ancillary features of games: the home team, whether or not the teams are conference opponents, March Madness games, etc.

Pomeroy’s derived team statistics are the main draw of this dataset; he describes his methodology in more depth in his [ratings glossary](http://kenpom.com/blog/index.php/weblog/entry/ratings_glossary) and [four factors discussion](http://kenpom.com/blog/index.php/weblog/four_factors/). Pomeroy produces "tempo-free" statistics about college basketball teams, adjusting their reported statistics by the "pace" (roughly the number of possessions) of their games. This methodology recognizes that teams' playing styles often affect the speed of each game, and that raw statistics might favor quick (and potentially inefficient) playing styles while failing to accurately describe slower, efficient ones. The calculations that ultimately proved most consequential to our predictions are Pomeroy's measurements of offensive and defensive efficiency, derived from the ratio of their points scored (or allowed in the case of defense) per possession.

If the reader is interested, Pomeroy's team statistics dataset for the latest season is available free of charge at [KenPom.com](http://kenpom.com/).

### Retrieval & Processing

Season-by-season team statistics (in CSV format) are available back to 2002 with the purchase of a one-year subscription to Ken Pomeroy’s website. Those were trivial to retrieve for our time range, as they are aggregated at the year level and can be easily downloaded by hand. Game records proved more complex to retrieve, as they are displayed on the website but not made available for easy download. To retrieve game outcomes, we used Python’s `requests` library to request raw HTML pages that contain team information. As this data is only available to premium subscribers, we used "spoofed" browser cookies (from a logged-in session) to make our requests, ultimately scraping more than 2,000 team information pages for analysis.

With raw HTML data in-hand, we opted to use [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/) to turn HTML tables with game outcomes into a CSV dataset. This process created one obvious complication: every game appeared twice in our dataset, with teams swapping roles as the “primary” team and the opponent. For example, since Duke and Wisconsin played one another, each of their team pages shows a game against the other. To avoid potentially compromising our analysis with these duplicate observations, we identified all pairs of games and randomly sampled one game from each pair (thus designating one team the "primary" team for which we predict a win or loss).

Finally, we merged game data with team statistics, with each game being augmented with statistics for both the primary and secondary teams. This created one additional challenge: every game featuring a given team would feature the same team statistics. Though a typical season has more than 10,000 games, the actual set of team statistics would be much smaller. In response, we also generated pairwise differences and ratios of team statistics between the primary and secondary team, thus ensuring diversity in our dataset and creating a features that *compare* both teams, rather than simply describing each team individually. A sample of the resultant dataset (the 2015 Final Four games) appears below; scroll to the right to see all fields:

<div style="max-height:1000px;max-width:1500px;overflow:auto;font-size:.8em">
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>game_id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</th>
      <th>game_group</th>
      <th>year</th>
      <th>date&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</th>
      <th>team</th>
      <th>opponent&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</th>
      <th>conference</th>
      <th>conference_tournament</th>
      <th>ncaa_tournament</th>
      <th>other_tournament</th>
      <th>location_Away</th>
      <th>location_Home</th>
      <th>location_Neutral</th>
      <th>location_SemiAway</th>
      <th>location_SemiHome</th>
      <th>team_Tempo</th>
      <th>team_RankTempo</th>
      <th>team_AdjTempo</th>
      <th>team_RankAdjTempo</th>
      <th>team_OE</th>
      <th>team_RankOE</th>
      <th>team_AdjOE</th>
      <th>team_RankAdjOE</th>
      <th>team_DE</th>
      <th>team_RankDE</th>
      <th>team_AdjDE</th>
      <th>team_RankAdjDE</th>
      <th>team_Pythag</th>
      <th>team_RankPythag</th>
      <th>opponent_Tempo</th>
      <th>opponent_RankTempo</th>
      <th>opponent_AdjTempo</th>
      <th>opponent_RankAdjTempo</th>
      <th>opponent_OE</th>
      <th>opponent_RankOE</th>
      <th>opponent_AdjOE</th>
      <th>opponent_RankAdjOE</th>
      <th>opponent_DE</th>
      <th>opponent_RankDE</th>
      <th>opponent_AdjDE</th>
      <th>opponent_RankAdjDE</th>
      <th>opponent_Pythag</th>
      <th>opponent_RankPythag</th>
      <th>diff_Tempo</th>
      <th>diff_RankTempo</th>
      <th>diff_AdjTempo</th>
      <th>diff_RankAdjTempo</th>
      <th>diff_OE</th>
      <th>diff_RankOE</th>
      <th>diff_AdjOE</th>
      <th>diff_RankAdjOE</th>
      <th>diff_DE</th>
      <th>diff_RankDE</th>
      <th>diff_AdjDE</th>
      <th>diff_RankAdjDE</th>
      <th>diff_Pythag</th>
      <th>diff_RankPythag</th>
      <th>ratio_Tempo</th>
      <th>ratio_RankTempo</th>
      <th>ratio_AdjTempo</th>
      <th>ratio_RankAdjTempo</th>
      <th>ratio_OE</th>
      <th>ratio_RankOE</th>
      <th>ratio_AdjOE</th>
      <th>ratio_RankAdjOE</th>
      <th>ratio_DE</th>
      <th>ratio_RankDE</th>
      <th>ratio_AdjDE</th>
      <th>ratio_RankAdjDE</th>
      <th>ratio_Pythag</th>
      <th>ratio_RankPythag</th>
      <th>points_for</th>
      <th>points_against</th>
      <th>win</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>20150404-duke-michiganst</td>
      <td>1</td>
      <td>2015</td>
      <td>2015-04-04</td>
      <td>Duke</td>
      <td>Michigan St.</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>65.9330</td>
      <td>120</td>
      <td>65.9619</td>
      <td>114</td>
      <td>119.6530</td>
      <td>3</td>
      <td>121.5639</td>
      <td>3</td>
      <td>96.6318</td>
      <td>51</td>
      <td>92.3456</td>
      <td>12</td>
      <td>0.959355</td>
      <td>4</td>
      <td>63.0403</td>
      <td>271</td>
      <td>63.5905</td>
      <td>245</td>
      <td>109.8856</td>
      <td>33</td>
      <td>114.5700</td>
      <td>15</td>
      <td>98.4175</td>
      <td>86</td>
      <td>95.5169</td>
      <td>47</td>
      <td>0.890088</td>
      <td>15</td>
      <td>2.8927</td>
      <td>-151</td>
      <td>2.3714</td>
      <td>-131</td>
      <td>9.7674</td>
      <td>-30</td>
      <td>6.9939</td>
      <td>-12</td>
      <td>-1.7857</td>
      <td>-35</td>
      <td>-3.1713</td>
      <td>-35</td>
      <td>0.069267</td>
      <td>-11</td>
      <td>1.045887</td>
      <td>0.442804</td>
      <td>1.037292</td>
      <td>0.465306</td>
      <td>1.088887</td>
      <td>0.090909</td>
      <td>1.061045</td>
      <td>0.2</td>
      <td>0.981856</td>
      <td>0.593023</td>
      <td>0.966799</td>
      <td>0.255319</td>
      <td>1.077820</td>
      <td>0.266667</td>
      <td>81</td>
      <td>61</td>
      <td>1</td>
    </tr>
    <tr>
      <td>20150404-kentucky-wisconsin</td>
      <td>1</td>
      <td>2015</td>
      <td>2015-04-04</td>
      <td>Wisconsin</td>
      <td>Kentucky</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>59.4998</td>
      <td>344</td>
      <td>59.0178</td>
      <td>346</td>
      <td>121.1329</td>
      <td>1</td>
      <td>127.8751</td>
      <td>1</td>
      <td>97.5034</td>
      <td>72</td>
      <td>96.3347</td>
      <td>54</td>
      <td>0.962927</td>
      <td>3</td>
      <td>63.7584</td>
      <td>241</td>
      <td>63.4728</td>
      <td>251</td>
      <td>115.4396</td>
      <td>9</td>
      <td>119.2883</td>
      <td>5</td>
      <td>84.6510</td>
      <td>1</td>
      <td>86.5378</td>
      <td>2</td>
      <td>0.975662</td>
      <td>1</td>
      <td>-4.2586</td>
      <td>103</td>
      <td>-4.4550</td>
      <td>95</td>
      <td>5.6933</td>
      <td>-8</td>
      <td>8.5868</td>
      <td>-4</td>
      <td>12.8524</td>
      <td>71</td>
      <td>9.7969</td>
      <td>52</td>
      <td>-0.012735</td>
      <td>2</td>
      <td>0.933207</td>
      <td>1.427386</td>
      <td>0.929812</td>
      <td>1.378486</td>
      <td>1.049318</td>
      <td>0.111111</td>
      <td>1.071984</td>
      <td>0.2</td>
      <td>1.151828</td>
      <td>72.000000</td>
      <td>1.113209</td>
      <td>27.000000</td>
      <td>0.986947</td>
      <td>3.000000</td>
      <td>71</td>
      <td>64</td>
      <td>1</td>
    </tr>
    <tr>
      <td>20150406-duke-wisconsin</td>
      <td>1</td>
      <td>2015</td>
      <td>2015-04-06</td>
      <td>Duke</td>
      <td>Wisconsin</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>65.9330</td>
      <td>120</td>
      <td>65.9619</td>
      <td>114</td>
      <td>119.6530</td>
      <td>3</td>
      <td>121.5639</td>
      <td>3</td>
      <td>96.6318</td>
      <td>51</td>
      <td>92.3456</td>
      <td>12</td>
      <td>0.959355</td>
      <td>4</td>
      <td>59.4998</td>
      <td>344</td>
      <td>59.0178</td>
      <td>346</td>
      <td>121.1329</td>
      <td>1</td>
      <td>127.8751</td>
      <td>1</td>
      <td>97.5034</td>
      <td>72</td>
      <td>96.3347</td>
      <td>54</td>
      <td>0.962927</td>
      <td>3</td>
      <td>6.4332</td>
      <td>-224</td>
      <td>6.9441</td>
      <td>-232</td>
      <td>-1.4799</td>
      <td>2</td>
      <td>-6.3112</td>
      <td>2</td>
      <td>-0.8716</td>
      <td>-21</td>
      <td>-3.9891</td>
      <td>-42</td>
      <td>-0.003572</td>
      <td>1</td>
      <td>1.108121</td>
      <td>0.348837</td>
      <td>1.117661</td>
      <td>0.329480</td>
      <td>0.987783</td>
      <td>3.000000</td>
      <td>0.950646</td>
      <td>3.0</td>
      <td>0.991061</td>
      <td>0.708333</td>
      <td>0.958591</td>
      <td>0.222222</td>
      <td>0.996290</td>
      <td>1.333333</td>
      <td>68</td>
      <td>63</td>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

Columns starting with `team_`, `opponent_`, `diff_`, and `tempo_` respresent KenPom-calculated and -derived features. Game outcome information appears in the final three columns. Columns ending with "OE" and "DE" represent the efficienty metrics described above.